# Data Transformation and ETL Pipeline
## Azure Synapse Analytics Data Platform (ASADP)

This notebook demonstrates advanced data transformation and ETL processes using Apache Spark in Azure Synapse Analytics.

### Features:
- **Medallion Architecture**: Bronze → Silver → Gold data layers
- **Delta Lake Integration**: ACID transactions and time travel
- **Data Quality Validation**: Comprehensive data quality checks
- **Schema Evolution**: Automatic schema management
- **Performance Optimization**: Partitioning and Z-ordering

### Prerequisites:
- Azure Synapse Spark Pool configured
- Data Lake Storage Gen2 with sample data
- Delta Lake libraries installed

In [ ]:
# Import required libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import *
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import json

# Initialize Spark session with Delta Lake support
spark = SparkSession.builder \
    .appName("ASADP-ETL-Pipeline") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

# Set configuration for better performance
spark.conf.set("spark.sql.adaptive.enabled", "true")
spark.conf.set("spark.sql.adaptive.coalescePartitions.enabled", "true")
spark.conf.set("spark.sql.adaptive.skewJoin.enabled", "true")

print(f"Spark version: {spark.version}")
print(f"Delta Lake version: {spark.sql('SELECT delta_version()').collect()[0][0]}")

In [ ]:
# Configuration
STORAGE_ACCOUNT = "your_storage_account_name"
CONTAINER_RAW = "raw"
CONTAINER_PROCESSED = "processed"
CONTAINER_CURATED = "curated"

# Data Lake paths
BASE_PATH = f"abfss://{CONTAINER_RAW}@{STORAGE_ACCOUNT}.dfs.core.windows.net"
BRONZE_PATH = f"abfss://{CONTAINER_RAW}@{STORAGE_ACCOUNT}.dfs.core.windows.net/bronze"
SILVER_PATH = f"abfss://{CONTAINER_PROCESSED}@{STORAGE_ACCOUNT}.dfs.core.windows.net/silver"
GOLD_PATH = f"abfss://{CONTAINER_CURATED}@{STORAGE_ACCOUNT}.dfs.core.windows.net/gold"

print(f"Bronze Layer: {BRONZE_PATH}")
print(f"Silver Layer: {SILVER_PATH}")
print(f"Gold Layer: {GOLD_PATH}")

## 1. Bronze Layer - Raw Data Ingestion

The Bronze layer contains raw, unprocessed data from various sources.

In [ ]:
# Generate sample sales data for demonstration
def generate_sample_sales_data(num_records=10000):
    """
    Generate sample sales data for ETL demonstration
    """
    from datetime import datetime, timedelta
    import random
    
    # Sample data generation
    customers = [f"CUST_{i:06d}" for i in range(1, 1001)]
    products = [f"PROD_{i:04d}" for i in range(1, 101)]
    regions = ["North", "South", "East", "West", "Central"]
    categories = ["Electronics", "Clothing", "Home", "Sports", "Books"]
    
    data = []
    base_date = datetime(2024, 1, 1)
    
    for i in range(num_records):
        record = {
            "transaction_id": f"TXN_{i+1:08d}",
            "customer_id": random.choice(customers),
            "product_id": random.choice(products),
            "product_category": random.choice(categories),
            "region": random.choice(regions),
            "quantity": random.randint(1, 10),
            "unit_price": round(random.uniform(10.0, 500.0), 2),
            "discount_percent": random.choice([0, 5, 10, 15, 20]),
            "transaction_date": (base_date + timedelta(days=random.randint(0, 365))).strftime("%Y-%m-%d"),
            "transaction_timestamp": (base_date + timedelta(days=random.randint(0, 365), 
                                                          hours=random.randint(0, 23), 
                                                          minutes=random.randint(0, 59))).isoformat(),
            "sales_rep_id": f"REP_{random.randint(1, 50):03d}",
            "channel": random.choice(["Online", "Store", "Phone", "Mobile App"]),
            "payment_method": random.choice(["Credit Card", "Debit Card", "Cash", "PayPal"])
        }
        data.append(record)
    
    return data

# Generate sample data
print("Generating sample sales data...")
sample_data = generate_sample_sales_data(50000)

# Create DataFrame
sales_df = spark.createDataFrame(sample_data)

print(f"Generated {sales_df.count()} sample records")
sales_df.printSchema()
sales_df.show(5)

In [ ]:
# Write to Bronze layer (Delta format)
print("Writing data to Bronze layer...")

bronze_table_path = f"{BRONZE_PATH}/sales_transactions"

# Add metadata columns for data lineage
bronze_df = sales_df \
    .withColumn("ingestion_timestamp", current_timestamp()) \
    .withColumn("source_system", lit("sample_generator")) \
    .withColumn("file_name", lit("generated_data")) \
    .withColumn("data_quality_score", lit(1.0))

# Write to Delta table with partitioning
bronze_df.write \
    .format("delta") \
    .mode("overwrite") \
    .partitionBy("transaction_date") \
    .option("mergeSchema", "true") \
    .save(bronze_table_path)

print(f"Bronze layer data written to: {bronze_table_path}")

# Create Delta table for SQL access
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS bronze_sales_transactions
    USING DELTA
    LOCATION '{bronze_table_path}'
""")

print("Bronze layer table created successfully")

## 2. Silver Layer - Data Cleansing and Validation

The Silver layer contains cleaned, validated, and enriched data.

In [ ]:
# Data Quality Functions
def validate_data_quality(df):
    """
    Perform comprehensive data quality validation
    """
    quality_checks = {}
    
    # Check for null values
    null_counts = {}
    for col in df.columns:
        null_count = df.filter(col(col).isNull()).count()
        null_counts[col] = null_count
    
    quality_checks['null_counts'] = null_counts
    
    # Check for duplicates
    total_records = df.count()
    distinct_records = df.distinct().count()
    duplicate_count = total_records - distinct_records
    
    quality_checks['total_records'] = total_records
    quality_checks['distinct_records'] = distinct_records
    quality_checks['duplicate_count'] = duplicate_count
    
    # Check data ranges
    quality_checks['quantity_stats'] = df.select(
        min("quantity").alias("min_quantity"),
        max("quantity").alias("max_quantity"),
        avg("quantity").alias("avg_quantity")
    ).collect()[0].asDict()
    
    quality_checks['price_stats'] = df.select(
        min("unit_price").alias("min_price"),
        max("unit_price").alias("max_price"),
        avg("unit_price").alias("avg_price")
    ).collect()[0].asDict()
    
    return quality_checks

# Read from Bronze layer
print("Reading data from Bronze layer...")
bronze_df = spark.read.format("delta").load(bronze_table_path)

# Perform data quality validation
print("Performing data quality validation...")
quality_report = validate_data_quality(bronze_df)

print("Data Quality Report:")
print(f"Total Records: {quality_report['total_records']}")
print(f"Distinct Records: {quality_report['distinct_records']}")
print(f"Duplicate Count: {quality_report['duplicate_count']}")
print(f"Quantity Stats: {quality_report['quantity_stats']}")
print(f"Price Stats: {quality_report['price_stats']}")

In [ ]:
# Data Cleansing and Transformation
print("Performing data cleansing and transformation...")

# Clean and transform data
silver_df = bronze_df \
    .filter(col("quantity") > 0) \
    .filter(col("unit_price") > 0) \
    .filter(col("discount_percent").between(0, 100)) \
    .withColumn("gross_amount", col("quantity") * col("unit_price")) \
    .withColumn("discount_amount", col("gross_amount") * col("discount_percent") / 100) \
    .withColumn("net_amount", col("gross_amount") - col("discount_amount")) \
    .withColumn("transaction_year", year(col("transaction_date"))) \
    .withColumn("transaction_month", month(col("transaction_date"))) \
    .withColumn("transaction_quarter", quarter(col("transaction_date"))) \
    .withColumn("transaction_day_of_week", dayofweek(col("transaction_date"))) \
    .withColumn("is_weekend", when(col("transaction_day_of_week").isin([1, 7]), True).otherwise(False)) \
    .withColumn("price_category", 
                when(col("unit_price") < 50, "Low")
                .when(col("unit_price") < 200, "Medium")
                .otherwise("High")) \
    .withColumn("customer_segment", 
                when(col("net_amount") < 100, "Bronze")
                .when(col("net_amount") < 500, "Silver")
                .when(col("net_amount") < 1000, "Gold")
                .otherwise("Platinum")) \
    .withColumn("processed_timestamp", current_timestamp()) \
    .withColumn("data_quality_flag", lit("PASSED"))

# Add row hash for change detection
silver_df = silver_df.withColumn(
    "row_hash",
    sha2(concat_ws("|", 
                   col("transaction_id"),
                   col("customer_id"),
                   col("product_id"),
                   col("quantity"),
                   col("unit_price")), 256)
)

print(f"Silver layer records after cleansing: {silver_df.count()}")
silver_df.printSchema()
silver_df.show(5)

In [ ]:
# Write to Silver layer
print("Writing data to Silver layer...")

silver_table_path = f"{SILVER_PATH}/sales_transactions"

# Write to Delta table with optimization
silver_df.write \
    .format("delta") \
    .mode("overwrite") \
    .partitionBy("transaction_year", "transaction_month") \
    .option("mergeSchema", "true") \
    .option("optimizeWrite", "true") \
    .save(silver_table_path)

print(f"Silver layer data written to: {silver_table_path}")

# Create Delta table for SQL access
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS silver_sales_transactions
    USING DELTA
    LOCATION '{silver_table_path}'
""")

# Optimize Delta table
spark.sql(f"OPTIMIZE silver_sales_transactions ZORDER BY (customer_id, product_id)")

print("Silver layer table created and optimized successfully")

## 3. Gold Layer - Business-Ready Aggregations

The Gold layer contains business-ready, aggregated data for analytics and reporting.

In [ ]:
# Read from Silver layer
print("Reading data from Silver layer...")
silver_df = spark.read.format("delta").load(silver_table_path)

# Create business aggregations
print("Creating business aggregations...")

# 1. Daily Sales Summary
daily_sales = silver_df.groupBy(
    "transaction_date",
    "transaction_year",
    "transaction_month",
    "transaction_quarter"
).agg(
    count("transaction_id").alias("total_transactions"),
    countDistinct("customer_id").alias("unique_customers"),
    countDistinct("product_id").alias("unique_products"),
    sum("quantity").alias("total_quantity"),
    sum("gross_amount").alias("total_gross_amount"),
    sum("discount_amount").alias("total_discount_amount"),
    sum("net_amount").alias("total_net_amount"),
    avg("net_amount").alias("avg_transaction_value"),
    max("net_amount").alias("max_transaction_value"),
    min("net_amount").alias("min_transaction_value")
).withColumn("aggregation_level", lit("daily")) \
 .withColumn("created_timestamp", current_timestamp())

print(f"Daily sales aggregations: {daily_sales.count()} records")
daily_sales.show(5)

# 2. Product Performance Summary
product_performance = silver_df.groupBy(
    "product_id",
    "product_category",
    "transaction_year",
    "transaction_month"
).agg(
    count("transaction_id").alias("total_transactions"),
    countDistinct("customer_id").alias("unique_customers"),
    sum("quantity").alias("total_quantity_sold"),
    sum("net_amount").alias("total_revenue"),
    avg("net_amount").alias("avg_transaction_value"),
    avg("unit_price").alias("avg_unit_price"),
    avg("discount_percent").alias("avg_discount_percent")
).withColumn("aggregation_level", lit("product_monthly")) \
 .withColumn("created_timestamp", current_timestamp())

print(f"Product performance aggregations: {product_performance.count()} records")
product_performance.show(5)

# 3. Customer Segmentation Summary
customer_segmentation = silver_df.groupBy(
    "customer_id",
    "customer_segment",
    "transaction_year"
).agg(
    count("transaction_id").alias("total_transactions"),
    countDistinct("product_id").alias("unique_products_purchased"),
    sum("quantity").alias("total_quantity_purchased"),
    sum("net_amount").alias("total_spent"),
    avg("net_amount").alias("avg_transaction_value"),
    max("transaction_date").alias("last_purchase_date"),
    min("transaction_date").alias("first_purchase_date")
).withColumn("aggregation_level", lit("customer_yearly")) \
 .withColumn("created_timestamp", current_timestamp())

print(f"Customer segmentation aggregations: {customer_segmentation.count()} records")
customer_segmentation.show(5)

# 4. Regional Performance Summary
regional_performance = silver_df.groupBy(
    "region",
    "transaction_year",
    "transaction_quarter"
).agg(
    count("transaction_id").alias("total_transactions"),
    countDistinct("customer_id").alias("unique_customers"),
    countDistinct("product_id").alias("unique_products"),
    sum("net_amount").alias("total_revenue"),
    avg("net_amount").alias("avg_transaction_value")
).withColumn("aggregation_level", lit("regional_quarterly")) \
 .withColumn("created_timestamp", current_timestamp())

print(f"Regional performance aggregations: {regional_performance.count()} records")
regional_performance.show(5)

In [ ]:
# Write Gold layer aggregations
print("Writing aggregations to Gold layer...")

# Write daily sales summary
daily_sales_path = f"{GOLD_PATH}/daily_sales_summary"
daily_sales.write \
    .format("delta") \
    .mode("overwrite") \
    .partitionBy("transaction_year", "transaction_month") \
    .option("optimizeWrite", "true") \
    .save(daily_sales_path)

# Write product performance
product_performance_path = f"{GOLD_PATH}/product_performance"
product_performance.write \
    .format("delta") \
    .mode("overwrite") \
    .partitionBy("product_category", "transaction_year") \
    .option("optimizeWrite", "true") \
    .save(product_performance_path)

# Write customer segmentation
customer_segmentation_path = f"{GOLD_PATH}/customer_segmentation"
customer_segmentation.write \
    .format("delta") \
    .mode("overwrite") \
    .partitionBy("customer_segment", "transaction_year") \
    .option("optimizeWrite", "true") \
    .save(customer_segmentation_path)

# Write regional performance
regional_performance_path = f"{GOLD_PATH}/regional_performance"
regional_performance.write \
    .format("delta") \
    .mode("overwrite") \
    .partitionBy("region", "transaction_year") \
    .option("optimizeWrite", "true") \
    .save(regional_performance_path)

print("Gold layer aggregations written successfully")

In [ ]:
# Create Gold layer tables for SQL access
print("Creating Gold layer tables...")

# Create tables
tables_config = [
    ("gold_daily_sales_summary", daily_sales_path),
    ("gold_product_performance", product_performance_path),
    ("gold_customer_segmentation", customer_segmentation_path),
    ("gold_regional_performance", regional_performance_path)
]

for table_name, table_path in tables_config:
    spark.sql(f"""
        CREATE TABLE IF NOT EXISTS {table_name}
        USING DELTA
        LOCATION '{table_path}'
    """)
    
    # Optimize tables
    spark.sql(f"OPTIMIZE {table_name}")
    
    print(f"Created and optimized table: {table_name}")

print("All Gold layer tables created successfully")

## 4. Data Quality Monitoring and Alerting

In [ ]:
# Data Quality Monitoring
def create_data_quality_report():
    """
    Create comprehensive data quality report
    """
    report = {}
    
    # Bronze layer quality
    bronze_df = spark.read.format("delta").load(bronze_table_path)
    report['bronze'] = {
        'record_count': bronze_df.count(),
        'null_transaction_ids': bronze_df.filter(col("transaction_id").isNull()).count(),
        'duplicate_transaction_ids': bronze_df.count() - bronze_df.select("transaction_id").distinct().count(),
        'invalid_quantities': bronze_df.filter(col("quantity") <= 0).count(),
        'invalid_prices': bronze_df.filter(col("unit_price") <= 0).count()
    }
    
    # Silver layer quality
    silver_df = spark.read.format("delta").load(silver_table_path)
    report['silver'] = {
        'record_count': silver_df.count(),
        'data_quality_passed': silver_df.filter(col("data_quality_flag") == "PASSED").count(),
        'avg_transaction_value': silver_df.agg(avg("net_amount")).collect()[0][0],
        'total_revenue': silver_df.agg(sum("net_amount")).collect()[0][0]
    }
    
    # Gold layer quality
    daily_sales_df = spark.read.format("delta").load(daily_sales_path)
    report['gold'] = {
        'daily_summaries': daily_sales_df.count(),
        'date_range': {
            'min_date': daily_sales_df.agg(min("transaction_date")).collect()[0][0],
            'max_date': daily_sales_df.agg(max("transaction_date")).collect()[0][0]
        },
        'total_aggregated_revenue': daily_sales_df.agg(sum("total_net_amount")).collect()[0][0]
    }
    
    return report

# Generate quality report
print("Generating data quality report...")
quality_report = create_data_quality_report()

print("\n=== DATA QUALITY REPORT ===")
print(f"Bronze Layer:")
print(f"  - Records: {quality_report['bronze']['record_count']:,}")
print(f"  - Null Transaction IDs: {quality_report['bronze']['null_transaction_ids']}")
print(f"  - Duplicate Transaction IDs: {quality_report['bronze']['duplicate_transaction_ids']}")
print(f"  - Invalid Quantities: {quality_report['bronze']['invalid_quantities']}")
print(f"  - Invalid Prices: {quality_report['bronze']['invalid_prices']}")

print(f"\nSilver Layer:")
print(f"  - Records: {quality_report['silver']['record_count']:,}")
print(f"  - Quality Passed: {quality_report['silver']['data_quality_passed']:,}")
print(f"  - Avg Transaction Value: ${quality_report['silver']['avg_transaction_value']:.2f}")
print(f"  - Total Revenue: ${quality_report['silver']['total_revenue']:,.2f}")

print(f"\nGold Layer:")
print(f"  - Daily Summaries: {quality_report['gold']['daily_summaries']}")
print(f"  - Date Range: {quality_report['gold']['date_range']['min_date']} to {quality_report['gold']['date_range']['max_date']}")
print(f"  - Total Aggregated Revenue: ${quality_report['gold']['total_aggregated_revenue']:,.2f}")

## 5. Performance Optimization and Maintenance

In [ ]:
# Performance optimization
print("Performing performance optimization...")

# Optimize all Delta tables
tables_to_optimize = [
    "bronze_sales_transactions",
    "silver_sales_transactions",
    "gold_daily_sales_summary",
    "gold_product_performance",
    "gold_customer_segmentation",
    "gold_regional_performance"
]

for table in tables_to_optimize:
    try:
        print(f"Optimizing table: {table}")
        spark.sql(f"OPTIMIZE {table}")
        
        # Update table statistics
        spark.sql(f"ANALYZE TABLE {table} COMPUTE STATISTICS")
        
        print(f"  ✓ Optimized {table}")
    except Exception as e:
        print(f"  ✗ Error optimizing {table}: {str(e)}")

print("Performance optimization completed")

In [ ]:
# Cleanup and maintenance
print("Performing cleanup and maintenance...")

# Vacuum old files (removes files older than retention period)
table_paths = [
    bronze_table_path,
    silver_table_path,
    daily_sales_path,
    product_performance_path,
    customer_segmentation_path,
    regional_performance_path
]

for path in table_paths:
    try:
        print(f"Vacuuming: {path}")
        # Note: In production, be careful with vacuum - it permanently deletes files
        # spark.sql(f"VACUUM delta.`{path}` RETAIN 168 HOURS")  # 7 days retention
        print(f"  ✓ Would vacuum {path} (skipped for safety)")
    except Exception as e:
        print(f"  ✗ Error vacuuming {path}: {str(e)}")

print("Cleanup and maintenance completed")

## 6. ETL Pipeline Summary and Next Steps

In [ ]:
# Pipeline execution summary
print("\n" + "="*60)
print("ETL PIPELINE EXECUTION SUMMARY")
print("="*60)

print("\n✅ COMPLETED SUCCESSFULLY:")
print("   • Bronze Layer: Raw data ingestion with metadata")
print("   • Silver Layer: Data cleansing and validation")
print("   • Gold Layer: Business-ready aggregations")
print("   • Data Quality: Comprehensive validation and monitoring")
print("   • Performance: Table optimization and statistics")
print("   • Maintenance: Cleanup and housekeeping")

print("\n📊 DATA LAYERS CREATED:")
print(f"   • Bronze: {bronze_table_path}")
print(f"   • Silver: {silver_table_path}")
print(f"   • Gold: {GOLD_PATH}/[multiple_aggregations]")

print("\n🔍 AVAILABLE TABLES:")
tables = spark.sql("SHOW TABLES").collect()
for table in tables:
    if 'sales' in table.tableName or 'gold' in table.tableName or 'bronze' in table.tableName or 'silver' in table.tableName:
        print(f"   • {table.tableName}")

print("\n🚀 NEXT STEPS:")
print("   1. Set up automated pipeline scheduling")
print("   2. Implement real-time streaming ingestion")
print("   3. Create Power BI reports and dashboards")
print("   4. Set up data quality alerts and monitoring")
print("   5. Implement machine learning models")
print("   6. Configure data governance and security")

print("\n" + "="*60)
print("ETL Pipeline completed successfully! 🎉")
print("="*60)